# Lorde - Melodrama Project

In [ ]:
import bearer
import requests
from bs4 import BeautifulSoup

In [ ]:
base_url = "https://api.genius.com"
headers = {'Authorization': bearer.TOKEN}

In [ ]:
def lyrics_from_song_api_path(song_api_path):
  song_url = base_url + song_api_path
  response = requests.get(song_url, headers=headers)
  json = response.json()
  path = json["response"]["song"]["path"]
  #gotta go regular html scraping... come on Genius
  page_url = "http://genius.com" + path
  page = requests.get(page_url)
  html = BeautifulSoup(page.text, "html.parser")
  #remove script tags that they put in the middle of the lyrics
  [h.extract() for h in html('script')]
  #at least Genius is nice and has a tag called 'lyrics'!
  lyrics = html.find("div", class_='lyrics').get_text() #updated css where the lyrics are based in HTML
  return lyrics

In [51]:
album_url = 'https://genius.com/albums/Lorde/Melodrama'
artist_name = 'Lorde'

In [52]:
page = requests.get(album_url)
track_soup = BeautifulSoup(page.text)
tracks = [i.get_text('*', strip=True) for i in track_soup.find_all('h3', class_='chart_row-content-title')]
tracks = tracks[:len(tracks)-1]
for i in range(len(tracks)):
  tracks[i] = tracks[i].split('*')[0]
tracks

[u'Green Light',
 u'Sober',
 u'Homemade Dynamite',
 u'The Louvre',
 u'Liability',
 u'Hard Feelings/Loveless',
 u'Sober II (Melodrama)',
 u'Writer in the Dark',
 u'Supercut',
 u'Liability (Reprise)',
 u'Perfect Places']

In [57]:
search_url = base_url + "/search"
album_lyrics = {}
for song_title in tracks:
  data = {'q': song_title}
  response = requests.get(search_url, data=data, headers=headers)
  json = response.json()
  song_info = None
  for hit in json["response"]["hits"]:
    if hit["result"]["primary_artist"]["name"] == artist_name:
      song_info = hit
      break
  if song_info:
    song_api_path = song_info["result"]["api_path"]
    album_lyrics[song_title] = lyrics_from_song_api_path(song_api_path)

In [58]:
album_lyrics

{u'Green Light': u"\n\n[Verse 1]\nI do my makeup in somebody else's car\nWe order different drinks at the same bars\nI know about what you did and I wanna scream the truth\nShe thinks you love the beach, you're such a damn liar\n\n[Refrain]\nWell those great whites, they have big teeth\nHope they bite you\nThought you said that you would always be in love\nBut you're not in love no more\nDid it frighten you\nHow we kissed when we danced on the light up floor?\nOn the light up floor\n\n[Pre-Chorus]\nBut I hear sounds in my mind\nBrand new sounds in my mind\nBut honey I'll be seein' you wherever I go\nBut honey I'll be seein' you down every road\nI'm waiting for it, that green light, I want it\n\n[Chorus]\n'Cause honey I'll come get my things, but I can't let go\nI'm waiting for it, that green light, I want it\nOh, I wish I could get my things and just let go\nI'm waiting for it, that green light, I want it\n\n[Verse 2]\nSometimes I wake up in a different bedroom\nI whisper things, the c